In [1]:
import re

def parse_syllable_file(syllable_file):
    syllables = []
    with open(syllable_file, 'r') as f:
        for line in f:
            syllables.append([syllable.split() for syllable in line.strip().split('.')])
    return syllables

def parse_phoneme_file(phoneme_file):
    phonemes = []
    with open(phoneme_file, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            phonemes.append((parts[0], float(parts[1]), float(parts[2])))
    return phonemes

def normalize_phoneme(phoneme):
    """Removes numerical markers from phoneme names (e.g., IH1 -> IH)."""
    return re.sub(r'\d', '', phoneme)

def extract_stress(phoneme):
    """Extracts the stress marker (1, 0, etc.) from the phoneme."""
    match = re.search(r'\d', phoneme)
    return int(match.group()) if match else None

def contains_vowel(phoneme):
    """Checks if the phoneme is a vowel."""
    vowels = {'A', 'E', 'I', 'O', 'U'}
    return any(v in normalize_phoneme(phoneme).upper() for v in vowels)

def generate_syllable_timestamps(syllables, phonemes):
    results = []
    phoneme_index = 0

    for word in syllables:
        word_syllables = []
        vowel_phonemes = []
        stress_labels = []

        for syllable in word:
            start_time = phonemes[phoneme_index][1]
            end_time = start_time
            vowel_found = None
            stress_found = None

            for phoneme in syllable:
                normalized_phoneme = normalize_phoneme(phoneme)
                while phoneme_index < len(phonemes) and normalize_phoneme(phonemes[phoneme_index][0]) != normalized_phoneme:
                    phoneme_index += 1

                if phoneme_index >= len(phonemes):
                    raise ValueError(f"Phoneme '{phoneme}' not found in phoneme file.")

                end_time = phonemes[phoneme_index][2]

                # Capture only the first vowel in the syllable
                if contains_vowel(phoneme) and vowel_found is None:
                    vowel_found = (phoneme, phonemes[phoneme_index][1], phonemes[phoneme_index][2])
                    stress_found = extract_stress(phonemes[phoneme_index][0])

                phoneme_index += 1

            word_syllables.append((" ".join(syllable), start_time, end_time))
            if vowel_found:
                vowel_phonemes.append(vowel_found)
                stress_labels.append(stress_found)

        results.append((word_syllables, vowel_phonemes, stress_labels))
    return results

def write_csv(output_file, filename, syllable_results):
    with open(output_file, 'w') as f:
        f.write("filename,syllables,vowels,stress\n")
        for i, (syllables, vowels, stress) in enumerate(syllable_results):
            f.write(f"{filename}_{i + 1},{syllables},{vowels},{stress}\n")

def main(syllable_file, phoneme_file, output_file, filename):
    syllables = parse_syllable_file(syllable_file)
    phonemes = parse_phoneme_file(phoneme_file)
    syllable_results = generate_syllable_timestamps(syllables, phonemes)
    write_csv(output_file, filename, syllable_results)

# # Replace these file paths with your actual file paths
# syllable_file = 'syllable_file.txt'
# phoneme_file = 'phoneme_file.txt'
# output_file = 'output.csv'
# filename = 'LJ001-0001'

# main(syllable_file, phoneme_file, output_file, filename)


In [3]:
# Replace these file paths with your actual file paths
syllable_file = '/home/monica/Desktop/Speech/project/p2tk-code-r18-python-syllabify(1)/syllables/LJ001-0001_phonemes_only_syllabified.txt'
phoneme_file = '/home/monica/Desktop/Speech/project/project_files/output_phonemes/LJ001-0001_phoneme_sequence.txt'
output_file = 'LJ001-0001.csv'
filename = 'LJ001-0001_phonemes_only_syllabified'

main(syllable_file, phoneme_file, output_file, filename)

In [5]:
import os
import re
import sys

def parse_syllable_file(syllable_file):
    syllables = []
    with open(syllable_file, 'r') as f:
        for line in f:
            syllables.append([syllable.split() for syllable in line.strip().split('.')])
    return syllables

def parse_phoneme_file(phoneme_file):
    phonemes = []
    with open(phoneme_file, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            phonemes.append((parts[0], float(parts[1]), float(parts[2])))
    return phonemes

def normalize_phoneme(phoneme):
    """Removes numerical markers from phoneme names (e.g., IH1 -> IH)."""
    return re.sub(r'\d', '', phoneme)

def extract_stress(phoneme):
    """Extracts the stress marker (1, 0, etc.) from the phoneme."""
    match = re.search(r'\d', phoneme)
    return int(match.group()) if match else None

def contains_vowel(phoneme):
    """Checks if the phoneme is a vowel."""
    vowels = {'A', 'E', 'I', 'O', 'U'}
    return any(v in normalize_phoneme(phoneme).upper() for v in vowels)

def generate_syllable_timestamps(syllables, phonemes):
    results = []
    phoneme_index = 0

    for word in syllables:
        word_syllables = []
        vowel_phonemes = []
        stress_labels = []

        for syllable in word:
            start_time = phonemes[phoneme_index][1] if phoneme_index < len(phonemes) else 0
            end_time = start_time
            vowel_found = None
            stress_found = None

            for phoneme in syllable:
                normalized_phoneme = normalize_phoneme(phoneme)
                while phoneme_index < len(phonemes) and normalize_phoneme(phonemes[phoneme_index][0]) != normalized_phoneme:
                    phoneme_index += 1

                if phoneme_index >= len(phonemes):
                    break  # Exit the loop if we run out of phonemes

                end_time = phonemes[phoneme_index][2]

                # Capture only the first vowel in the syllable
                if contains_vowel(phoneme) and vowel_found is None:
                    vowel_found = (phoneme, phonemes[phoneme_index][1], phonemes[phoneme_index][2])
                    stress_found = extract_stress(phonemes[phoneme_index][0])

                phoneme_index += 1

            word_syllables.append((" ".join(syllable), start_time, end_time))
            if vowel_found:
                vowel_phonemes.append(vowel_found)
                stress_labels.append(stress_found)

        results.append((word_syllables, vowel_phonemes, stress_labels))
    return results

def write_csv(output_file, filename, syllable_results):
    with open(output_file, 'w') as f:
        f.write("filename,syllables,vowels,stress\n")
        filename = filename.split('_')[0]+'.wav'
        for i, (syllables, vowels, stress) in enumerate(syllable_results):
            
            f.write(f'"{filename}",{syllables},{vowels},{stress}\n')

def process_files(syllable_folder, phoneme_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    syllable_files = sorted(os.listdir(syllable_folder))
    phoneme_files = sorted(os.listdir(phoneme_folder))

    for syllable_file, phoneme_file in zip(syllable_files, phoneme_files):
        syllable_path = os.path.join(syllable_folder, syllable_file)
        phoneme_path = os.path.join(phoneme_folder, phoneme_file)
        output_file = os.path.join(output_folder, f"{os.path.splitext(syllable_file)[0]}.csv")
        filename = os.path.splitext(syllable_file)[0]

        syllables = parse_syllable_file(syllable_path)
        phonemes = parse_phoneme_file(phoneme_path)
        
        # Print lengths for debugging
        print(f"Processing {syllable_file}: {len(syllables)} syllables, {len(phonemes)} phonemes")
        
        syllable_results = generate_syllable_timestamps(syllables, phonemes)
        write_csv(output_file, filename, syllable_results)

# Specify folder paths
syllable_folder = '/home/monica/Desktop/Speech/project/p2tk-code-r18-python-syllabify(1)/syllables'
phoneme_folder = '/home/monica/Desktop/Speech/project/project_files/output_phonemes'
output_folder = '/home/monica/Desktop/Speech/project/p2tk-code-r18-python-syllabify(1)/p2tk-code-r18-python-syllabify/output'

# Process all files in the folders
process_files(syllable_folder, phoneme_folder, output_folder)


Processing LJ001-0001_phonemes_only_syllabified.txt: 27 syllables, 107 phonemes
Processing LJ001-0002_phonemes_only_syllabified.txt: 4 syllables, 23 phonemes
Processing LJ001-0003_phonemes_only_syllabified.txt: 24 syllables, 103 phonemes
Processing LJ001-0004_phonemes_only_syllabified.txt: 14 syllables, 58 phonemes
Processing LJ001-0005_phonemes_only_syllabified.txt: 25 syllables, 101 phonemes
Processing LJ001-0006_phonemes_only_syllabified.txt: 14 syllables, 52 phonemes
Processing LJ001-0007_phonemes_only_syllabified.txt: 19 syllables, 79 phonemes
Processing LJ001-0008_phonemes_only_syllabified.txt: 4 syllables, 16 phonemes
Processing LJ001-0009_phonemes_only_syllabified.txt: 19 syllables, 69 phonemes
Processing LJ001-0010_phonemes_only_syllabified.txt: 18 syllables, 84 phonemes
Processing LJ001-0011_phonemes_only_syllabified.txt: 15 syllables, 48 phonemes
Processing LJ001-0012_phonemes_only_syllabified.txt: 17 syllables, 73 phonemes
Processing LJ001-0013_phonemes_only_syllabified.txt